In [7]:
from typing import List
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.embeddings import Embeddings
from langchain_core.documents import Document

class MiniLMEmbeddings(Embeddings):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def embed_documents(self, texts: List[str], batch_size=500) -> List[List[float]]:
        # Use batching to speed up the encoding
        return self.model.encode(texts, batch_size=batch_size, show_progress_bar=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

data = pd.read_csv("train_data_v2_57k.csv")
#data, validation = train_test_split(data, test_size=250)
validation = data.loc[val_indeces]
data = data.drop(val_indeces)
print(len(validation))
print(len(data))

# Filter only rows where 'en_title' is a string
filtered_data = data[data['en_title'].apply(lambda x: isinstance(x, str))]

# Vectorize the creation of Document objects
docs = [
    Document(
        page_content=row['en_title'],
        metadata={'en_title': row['en_title'], 'fr_title': row['fr_title']}
    )
    for _, row in filtered_data.iterrows()
]

vec_store = InMemoryVectorStore(MiniLMEmbeddings())
vec_store.add_documents(documents=docs)

250
56863


Batches: 100%|██████████| 114/114 [07:48<00:00,  4.11s/it]


['8d8ba935-3a35-4a30-b709-b65241e24f5c',
 '26b4a044-2028-4e96-b7ac-1463f5c08aa8',
 'd3ea45ac-42eb-4e63-9f2a-d4389f9d4161',
 '5abfac1b-6c19-488b-bf0d-122914883730',
 '64e1eb2e-2042-443c-afa3-2a23431c9516',
 'fd95ef9e-74e8-4c8c-98e1-4a06065b04be',
 '9449ce15-4a4f-44c8-a480-f7e15879cd2b',
 'e638486e-76dd-49d2-8db3-6bf88f137af1',
 '00953a51-904c-4c10-9795-a5bb8d25ecf9',
 '1b11d9b3-45f0-4ef7-a150-52939fdcd4bc',
 '122ab241-9991-4c48-b906-317b3788ca92',
 '1dd24b69-2755-4d79-814e-04488767a4ec',
 'e3eb6d31-9164-4cd6-849c-0dadf117a565',
 'b8c9c268-b1ee-4555-bbbc-104299261c9c',
 '5ccc24f3-7d26-4019-acfc-946ea35e9e11',
 '714ee86a-bcce-4cea-beb7-19f9d5ad8a7e',
 '2ce80372-7d2c-4dfb-8e29-5b122a42c767',
 '0b440459-ff0b-444d-83e5-66668e0305ce',
 'd1d1fbb1-6b17-47ef-b396-90e5ca3b3c51',
 '8d647a48-3875-4cee-932f-3d4f9300834d',
 '57123732-c8b9-4d1f-ade5-63bfe375decd',
 'd1c1c43d-f890-4d94-8653-3d4599f7c12a',
 'afcc55c8-69eb-4dff-b7ba-04192140d107',
 'a1674563-ea24-419b-8db9-97783215e917',
 '6f8a0eb6-d9bd-

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import openai
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.environ["LITELLM_KEY"]
base_url="http://10.2.47.37"

client = openai.OpenAI(
    api_key=api_key,
    base_url=base_url
)

def make_examples_section(docs):
    lines = []
    for doc in docs:
        lines.append(f"English: {doc.metadata['en_title']}")
        lines.append(f"French: {doc.metadata['fr_title']}\n")
    return "\n".join(lines)

def translate_with_few_shot(english_title, model="cheap-ian", k_neighbors=5, verbose=False):
    try:
        system_prompt_template = ("""You are a highly skilled translator specializing in technical and legal documents, with specific expertise in translating English patent titles into the most accurate and natural French equivalents. Always maintain the precise technical terminology and meaning of the original, adapting phrasing for French patent standards. Return only the title/word translated, match the casing, do not be conversational. Here are some examples:
                            \n{examples}""")
        prompt_template = ChatPromptTemplate.from_messages(
            [("system", system_prompt_template), ("user", "{text}")]
        )
        results = vec_store.similarity_search(query=english_title, k=k_neighbors)

        prompt = prompt_template.invoke({"examples":make_examples_section(results), "text":english_title})
        messages = prompt.to_messages()
        sys_msg = messages[0].content
        user_msg = messages[1].content

        if verbose:
            print("sys:")
            print(sys_msg)
            print("user:")
            print(user_msg)

        response = client.chat.completions.create(
            model=model,
            messages = [
                {
                    "role": "system",
                    "content": sys_msg
                },
                {
                    "role": "user",
                    "content": user_msg
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(english_title)
        print(e)
        return ""

def translate_with_base_model(english_title, model="cheap-ian"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages = [
                {
                    "role": "system",
                    "content": "You are a highly skilled translator specializing in technical and legal documents, with specific expertise in translating English patent titles into the most accurate and natural French equivalents. Always maintain the precise technical terminology and meaning of the original, adapting phrasing for French patent standards. Return only the title/word translated, match the casing, do not be conversational."
                },
                {
                    "role": "user",
                    "content": english_title
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(english_title)
        print(e)
        return ""

#val_row = validation.iloc[-100]
#print("Sample English: {} \nSample French: {}\n".format(val_row["en_title"], val_row["fr_title"]))
#translate_with_base_model(val_row["en_title"])   

In [ ]:
french_to_english = {
    "Bonjour": "Hello",
    "Maison": "House",
    "Chat": "Cat",
    "Livre": "Book",
    "École": "School",
    "Pomme": "Apple",
    "Chaise": "Chair",
    "Amour": "Love",
    "Eau": "Water",
    "Nuit": "Night"
}

for key, value in french_to_english.items():
    translation = translate_with_base_model(value, model="best")
    print(f"English: {value}, Translation: {translation}")

In [5]:
from bert_score import score
from rouge_score import rouge_scorer
import torch
import sacrebleu

device = 'cuda' if torch.cuda.is_available() else 'cpu'

r_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
chrf = sacrebleu.CHRF()

model = "best"
validation["fr_translate_base"] = validation["en_title"].apply(lambda x: translate_with_base_model(x, model=model))
validation["fr_translate_10-shot"] = validation["en_title"].apply(lambda x: translate_with_few_shot(x, model=model, k_neighbors=10))
validation["fr_translate_5-shot"] = validation["en_title"].apply(lambda x: translate_with_few_shot(x, model=model, k_neighbors=5))

IMPROVED STABILITY INSECTICIDAL COMPOSITIONS AND METHODS OF MAKING AND USING THE SAME
Error code: 400 - {'error': {'message': 'litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - Error code: 400 - {\'error\': {\'message\': "The response was filtered due to the prompt triggering Azure OpenAI\'s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", \'type\': None, \'param\': \'prompt\', \'code\': \'content_filter\', \'status\': 400, \'innererror\': {\'code\': \'ResponsibleAIPolicyViolation\', \'content_filter_result\': {\'hate\': {\'filtered\': False, \'severity\': \'safe\'}, \'jailbreak\': {\'filtered\': False, \'detected\': False}, \'self_harm\': {\'filtered\': False, \'severity\': \'low\'}, \'sexual\': {\'filtered\': False, \'severity\': \'safe\'}, \'violence\': {\'filtered\': True,

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


IMPROVED STABILITY INSECTICIDAL COMPOSITIONS AND METHODS OF MAKING AND USING THE SAME
Error code: 400 - {'error': {'message': 'litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - Error code: 400 - {\'error\': {\'message\': "The response was filtered due to the prompt triggering Azure OpenAI\'s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", \'type\': None, \'param\': \'prompt\', \'code\': \'content_filter\', \'status\': 400, \'innererror\': {\'code\': \'ResponsibleAIPolicyViolation\', \'content_filter_result\': {\'hate\': {\'filtered\': False, \'severity\': \'safe\'}, \'jailbreak\': {\'filtered\': False, \'detected\': False}, \'self_harm\': {\'filtered\': False, \'severity\': \'low\'}, \'sexual\': {\'filtered\': False, \'severity\': \'safe\'}, \'violence\': {\'filtered\': True,

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


LATERAL LOADING OF MICROCAPILLARY ARRAYS
Error code: 400 - {'error': {'message': 'litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - Error code: 400 - {\'error\': {\'message\': "The response was filtered due to the prompt triggering Azure OpenAI\'s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", \'type\': None, \'param\': \'prompt\', \'code\': \'content_filter\', \'status\': 400, \'innererror\': {\'code\': \'ResponsibleAIPolicyViolation\', \'content_filter_result\': {\'hate\': {\'filtered\': False, \'severity\': \'safe\'}, \'jailbreak\': {\'filtered\': False, \'detected\': False}, \'self_harm\': {\'filtered\': False, \'severity\': \'safe\'}, \'sexual\': {\'filtered\': False, \'severity\': \'safe\'}, \'violence\': {\'filtered\': True, \'severity\': \'medium\'}}}}}\nmodel=best. 

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


IMPROVED STABILITY INSECTICIDAL COMPOSITIONS AND METHODS OF MAKING AND USING THE SAME
Error code: 400 - {'error': {'message': 'litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - Error code: 400 - {\'error\': {\'message\': "The response was filtered due to the prompt triggering Azure OpenAI\'s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", \'type\': None, \'param\': \'prompt\', \'code\': \'content_filter\', \'status\': 400, \'innererror\': {\'code\': \'ResponsibleAIPolicyViolation\', \'content_filter_result\': {\'hate\': {\'filtered\': False, \'severity\': \'safe\'}, \'jailbreak\': {\'filtered\': False, \'detected\': False}, \'self_harm\': {\'filtered\': False, \'severity\': \'low\'}, \'sexual\': {\'filtered\': False, \'severity\': \'safe\'}, \'violence\': {\'filtered\': True,

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


LATERAL LOADING OF MICROCAPILLARY ARRAYS
Error code: 400 - {'error': {'message': 'litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - Error code: 400 - {\'error\': {\'message\': "The response was filtered due to the prompt triggering Azure OpenAI\'s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", \'type\': None, \'param\': \'prompt\', \'code\': \'content_filter\', \'status\': 400, \'innererror\': {\'code\': \'ResponsibleAIPolicyViolation\', \'content_filter_result\': {\'hate\': {\'filtered\': False, \'severity\': \'safe\'}, \'jailbreak\': {\'filtered\': False, \'detected\': False}, \'self_harm\': {\'filtered\': False, \'severity\': \'safe\'}, \'sexual\': {\'filtered\': False, \'severity\': \'safe\'}, \'violence\': {\'filtered\': True, \'severity\': \'medium\'}}}}}\nmodel=best. 

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


In [ ]:
def clean_output(input_text):
    try:
        text_without_asterisks = input_text.replace('**', '')
        text_without_asterisks = text_without_asterisks.replace('\n', '')
        french_index = text_without_asterisks.find('French:')
        if french_index != -1:
            text_after_french = text_without_asterisks[french_index + len('French:'):]
        else:
            text_after_french = text_without_asterisks
        french_index = text_without_asterisks.upper().find('French Translation:'.upper())
        if french_index != -1:
            text_after_french = text_after_french[french_index + len('French Translation:'):]
        else:
            text_after_french = text_after_french
        final_text = text_after_french.upper()
        return final_text
    except:
        return ""

In [55]:
from bert_score import score
from rouge_score import rouge_scorer
import torch
import sacrebleu
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

validation = pd.read_csv('validation-titles.csv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

r_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
chrf = sacrebleu.CHRF()

def validation_score_generate(validation, model_column, ref_column="fr_title", score_column="score", clean_output = lambda x: x):
    _, _, F1_base = score(validation[model_column].astype(str).apply(clean_output).tolist(), validation[ref_column].astype(str).tolist(), lang="fr", device=device, batch_size=300, verbose=True)
    c_scores_base = [chrf.sentence_score(translated, [ref]).score 
                 for translated, ref in zip(validation[model_column].astype(str).apply(clean_output), validation[ref_column].astype(str))]
    rouge_scores_base = [r_scorer.score(ref, hyp)['rougeL'].fmeasure*100
                     for ref, hyp in zip(validation[ref_column].astype(str), validation[model_column].astype(str).apply(clean_output))]
    validation[score_column] = [(0.2*c + 0.3*f*100 + 0.5*r).item()
                            for c, f, r in zip(c_scores_base, F1_base, rouge_scores_base)]
    return validation

def evaluate_translations_td_idf(df, model_column, ref_column="fr_title", score_column="score"):
    # Concatenate reference and translated texts for vectorization
    all_texts = df[ref_column].astype(str).tolist() + df[model_column].astype(str).tolist()
    
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Vectorize the texts
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    
    # Split the TF-IDF matrix into reference and model parts
    ref_tfidf = tfidf_matrix[:len(df)]
    model_tfidf = tfidf_matrix[len(df):]
    
    # Compute cosine similarity for each pair of reference/model texts
    similarities = []
    for ref_vec, model_vec in zip(ref_tfidf, model_tfidf):
        similarity = cosine_similarity(ref_vec, model_vec)[0][0]
        similarities.append(similarity)
    
    # Add similarity scores to the DataFrame
    df[score_column] = similarities
    return df

validation = validation_score_generate(validation, model_column="fr_translate_base", score_column="base_score")
validation = validation_score_generate(validation, model_column="fr_translate_5-shot", score_column="5-shot-score")
validation = validation_score_generate(validation, model_column="fr_translate_10-shot", score_column="10-shot-score")

print("1")
validation = evaluate_translations_td_idf(validation, model_column="fr_translate_base", score_column="tdidf-base_score")
print("2")
validation = evaluate_translations_td_idf(validation, model_column="fr_translate_5-shot", score_column="tdidf-5-shot-score")
print("3")
validation = evaluate_translations_td_idf(validation, model_column="fr_translate_10-shot", score_column="tdidf-10-shot-score")

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:29<00:00, 14.61s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


done in 29.92 seconds, 8.36 sentences/sec
calculating scores...
computing bert embedding.


...

done in 27.87 seconds, 8.97 sentences/sec
calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:27<00:00, 13.68s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00,  3.23it/s]


done in 27.68 seconds, 9.03 sentences/sec
1
2
3


In [56]:
score_ver = "tdidf-"
validation[validation[f"{score_ver}base_score"] != 0][f"{score_ver}base_score"].mean(),\
validation[validation[f"{score_ver}base_score"] != 0][f"{score_ver}base_score"].std(),\
validation[validation[f"{score_ver}5-shot-score"] != 0][f"{score_ver}5-shot-score"].mean(),\
validation[validation[f"{score_ver}5-shot-score"] != 0][f"{score_ver}5-shot-score"].std(),\
validation[validation[f"{score_ver}10-shot-score"] != 0][f"{score_ver}10-shot-score"].mean(),\
validation[validation[f"{score_ver}10-shot-score"] != 0][f"{score_ver}10-shot-score"].std()

(np.float64(0.5398217122589464),
 np.float64(0.24242584068227474),
 np.float64(0.758603202536415),
 np.float64(0.2574494185099043),
 np.float64(0.7710907004225526),
 np.float64(0.23923464259554916))

In [57]:
score_ver = ""
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}base_score"].mean(),\
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}base_score"].std(),\
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}5-shot-score"].mean(),\
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}5-shot-score"].std(),\
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}10-shot-score"].mean(),\
validation[validation[f"{score_ver}5-shot-score"] <= 98][f"{score_ver}10-shot-score"].std()

(np.float64(62.73595064700007),
 np.float64(17.34646759285099),
 np.float64(70.33966005633691),
 np.float64(19.399076882451055),
 np.float64(70.7026215913053),
 np.float64(21.2436231784896))

In [58]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_vecstore_similarity(english_title, model, k_neighbors=5):
    results = vec_store.similarity_search(query=english_title, k=k_neighbors)
    samples = [doc.metadata['en_title'] for doc in results]
    embeddings = np.array(model.encode(samples, batch_size=k_neighbors, show_progress_bar=True).tolist()).reshape(k_neighbors, 1, -1)
    embedding0 = np.array(model.encode([english_title], batch_size=k_neighbors, show_progress_bar=True).tolist()[0]).reshape(1, -1)
    similarities = pd.Series([cosine_similarity(embedding0, embedding)[0][0] for embedding in embeddings]).astype(float)
    return similarities.mean(), similarities.max(), similarities.min() 

# TODO: try other model embeddings (eg. document embeddings)
validation[['5-shot-sim-mean', '5-shot-sim-max', '5-shot-sim-min']] = validation['en_title'].apply(lambda title: pd.Series(get_vecstore_similarity(title, model)))

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


In [ ]:
import plotly.express as px
import statsmodels.api as sm
import pandas as pd

# Create a dataframe
df = pd.DataFrame()

# Define variables for the plotting
df['Max DB Cosine Similarity'] = (validation[(validation['5-shot-sim-max'] < 0.99)]['5-shot-sim-max'] * 100)
df['Translation Score'] = (validation[(validation['5-shot-sim-max'] < 0.99)]['5-shot-score'])

# Perform OLS regression to get the p-value
X = sm.add_constant(df['Max DB Cosine Similarity'])  # Adds intercept term
model = sm.OLS(df['Translation Score'], X).fit()
p_value = model.pvalues['Max DB Cosine Similarity']

# Display the scatter plot with trendline
fig = px.scatter(
    df,
    x='Max DB Cosine Similarity',
    y='Translation Score',
    title=f'Translation Score vs. Database Example Similarity (p-value: {p_value:.3e})',
    height=650,
    width=750,
    trendline='ols',
    trendline_color_override='red'
)

# Show the plot
fig.show()

print("P-value for the line of best fit: {}".format(p_value))

P-value for the line of best fit: 0.2986450931424334


In [73]:
from scipy.stats import pearsonr

pearsonr(df['x'], df['y'])

PearsonRResult(statistic=np.float64(0.17378195145740394), pvalue=np.float64(0.005869987652123395))

In [77]:
import pandas as pd
from bert_score import score
from rouge_score import rouge_scorer
import torch
import sacrebleu

device = 'cuda' if torch.cuda.is_available() else 'cpu'

gdata = pd.read_csv(r"google_translate.csv")

validation_df = validation[["en_title", "fr_title"]]
r_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
chrf = sacrebleu.CHRF()

def validation_score_generate(validation, model_column, ref_column="fr_title", score_column="score", clean_output = lambda x: x):
    _, _, F1_base = score(validation[model_column].apply(clean_output).tolist(), validation[ref_column].tolist(), lang="fr", device=device)
    c_scores_base = [chrf.sentence_score(translated, [ref]).score 
                 for translated, ref in zip(validation[model_column].apply(clean_output), validation[ref_column])]
    rouge_scores_base = [r_scorer.score(ref, hyp)['rougeL'].fmeasure*100
                     for ref, hyp in zip(validation[ref_column], validation[model_column].apply(clean_output))]
    validation[score_column] = [0.2*c + 0.3*f*100 + 0.5*r
                            for c, f, r in zip(c_scores_base, F1_base, rouge_scores_base)]
    print(c_scores_base[0])
    return validation

gdata = validation_score_generate(gdata, "fr_google_translate")
gdata["score"].astype(int).mean() 

61.688040542650114


np.float64(70.064)